In [1]:
import pandas as pd
import numpy as np

### Load in Cleaned Data

- Discuss Preprocessing and cleaning

In [4]:
trades=pd.read_csv('data/cleaned_trades.csv')
trades

,Unnamed: 0,SIP_Timestamp,Exchange,Symbol,Trade_Volume,Trade_Price,Sale_Condition,Source_of_Trade,Trade_Correction_Indicator,Sequence_Number,Trade_Id,Trade_Reporting_Facility,Trade_Through_Exempt_Indicator,date
0,2020-01-15 09:30:01.004096,2020-01-15 09:30:01.004475,N,AAPL,100,311.94,@F,N,0,25659,5,,1,2020-01-15
1,2020-01-15 09:30:01.100463,2020-01-15 09:30:01.100841,N,AAPL,3,311.91,@F I,N,0,26087,6,,1,2020-01-15
2,2020-01-15 09:30:01.438061,2020-01-15 09:30:01.438450,N,AAPL,10,311.89,@ I,N,0,26776,7,,0,2020-01-15
3,2020-01-15 09:30:01.438068,2020-01-15 09:30:01.438451,N,AAPL,46,311.89,@ I,N,0,26777,8,,0,2020-01-15
4,2020-01-15 09:30:01.677976,2020-01-15 09:30:01.678356,N,AAPL,1,311.95,@ I,N,0,27223,9,,0,2020-01-15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9989,2020-01-17 09:39:08.500600,2020-01-17 09:39:08.500981,N,AAPL,200,315.66,@F,N,0,139420,355,,1,2020-01-17
9990,2020-01-17 09:39:08.890641,2020-01-17 09:39:08.891019,N,AAPL,20,315.69,@ I,N,0,139541,356,,0,2020-01-17
9991,2020-01-17 09:39:09.152079,2020-01-17 09:39:09.152456,N,AAPL,100,315.66,@,N,0,139673,357,,0,2020-01-17
9992,2020-01-17 09:39:09.984181,2020-01-17 09:39:09.984559,N,AAPL,100,315.63,@F,N,0,139778,358,,1,2020-01-17


In [5]:
quotes=pd.read_csv('data/cleaned_quotes.csv')
quotes

,Unnamed: 0,SIP_Timestamp,Exchange,Symbol,Bid_Price,Bid_Size,Offer_Price,Offer_Size,Quote_Condition,Sequence_Number,Source_Of_Quote,Best_Bid_Exchange,Best_Bid_Price,Best_Bid_Size,Best_Offer_Exchange,Best_Offer_Price,Best_Offer_Size,LULD_NBBO_Indicator,Security_Status_Indicator,date
0,2020-01-15 09:30:00.953633,2020-01-15 09:30:00.954013,N,AAPL,311.94,1.0,326.18,1.0,R,356546,N,N,311.94,1.0,V,311.96,1.0,A,,2020-01-15
1,2020-01-15 09:30:01.004096,2020-01-15 09:30:01.004476,N,AAPL,310.85,1.0,326.18,1.0,R,357650,N,Q,311.92,2.0,Q,311.99,1.0,A,,2020-01-15
2,2020-01-15 09:30:01.076382,2020-01-15 09:30:01.076759,N,AAPL,310.85,1.0,311.96,1.0,R,358817,N,Q,311.89,3.0,N,311.96,1.0,A,,2020-01-15
3,2020-01-15 09:30:01.645062,2020-01-15 09:30:01.645440,N,AAPL,310.85,1.0,311.95,4.0,R,363970,N,Q,311.90,1.0,N,311.95,4.0,A,,2020-01-15
4,2020-01-15 09:30:01.677976,2020-01-15 09:30:01.678357,N,AAPL,310.85,1.0,311.95,3.0,R,364328,N,P,311.86,2.0,N,311.95,3.0,A,,2020-01-15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
754098,2020-01-29 15:59:37.990504,2020-01-29 15:59:37.990881,N,AAPL,324.10,1.0,324.60,1.0,R,31890276,N,Q,324.34,1.0,Q,324.39,1.0,A,,2020-01-29
754099,2020-01-29 15:59:42.591204,2020-01-29 15:59:42.591582,N,AAPL,324.05,3.0,324.28,1.0,R,31920111,N,Q,324.23,1.0,Q,324.28,1.0,A,,2020-01-29
754100,2020-01-29 15:59:51.441479,2020-01-29 15:59:51.441858,N,AAPL,323.97,1.0,324.27,1.0,R,31971787,N,Q,324.20,1.0,Q,324.27,2.0,A,,2020-01-29
754101,2020-01-29 15:59:52.555595,2020-01-29 15:59:52.555972,N,AAPL,323.97,1.0,324.23,1.0,R,31978303,N,Q,324.17,2.0,N,324.23,1.0,A,,2020-01-29
